<a href="https://colab.research.google.com/github/armandossrecife/piloto/blob/main/analise_designiteenterprise_cassandra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Checa ambiente

## Versão do Java

In [ ]:
!java --version

openjdk 11.0.19 2023-04-18
OpenJDK Runtime Environment (build 11.0.19+7-post-Ubuntu-0ubuntu120.04.1)
OpenJDK 64-Bit Server VM (build 11.0.19+7-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)


## Versão do Python

In [ ]:
!python3 --version

Python 3.10.12


# Carrega Funcões

In [2]:
import pandas as pd
import os
import requests
import re

def download_designite(url):
  try:
    filename = 'DesigniteJava.jar'  # replace with the desired filename to save the downloaded file
    response = requests.get(url)

    with open(filename, 'wb') as f:
        f.write(response.content)

    print(f'{filename} downloaded successfully!')
  except Exception as e:
    print(f'Erro: {str(e)}')

def checa_indice_language(relacao_linguagens):
  indice=-1
  for i, each in enumerate(relacao_linguagens):
    if "Language" in each:
      indice = i
      break
  return indice

def instala_cloc():
  try:
    comando = !sudo apt-get install cloc
    print(comando)
  except Exception as e:
    print(f'Erro: {str(e)}')

def instala_tree():
  try:
    comando = !sudo apt install tree
    print(comando)
  except Exception as ex:
    print(f'Erro ao instalar o comando tree: {str(ex)}')

def clona_repositorio(url):
  try:
    comando = f'git clone {url}'
    print(f'Clonando o repositório {url}')
    print(comando)
    os.system(comando)
    print('concluido com sucesso!')
  except Exception as e:
    print(f'Erro: {str(e)}')

def get_primeiro_commit(repositorio):
  try:
    os.environ['REPOSITORIO_ORIGINAL'] = repositorio
    # Primeiro commit
    comando = !cd $REPOSITORIO_ORIGINAL && git rev-list --all | tail -n 1
  except Exception as e:
    print(f'Erro: {str(e)}')
  return comando

def get_commit_tag(tag, repositorio):
  try:
    os.environ['TAG'] = tag
    os.environ['REPOSITORIO_ORIGINAL'] = repositorio
    comando = !cd $REPOSITORIO_ORIGINAL && git rev-list -n 1 $TAG
  except Exception as e:
    print(f'Erro: {str(e)}')
  return comando

def get_ultimo_commit(repositorio):
  try:
    os.environ['REPOSITORIO_ORIGINAL'] = repositorio
    # Primeiro commit
    comando = !cd $REPOSITORIO_ORIGINAL && git rev-list --all | head -n 1
  except Exception as e:
    print(f'Erro: {str(e)}')
  return comando

def cria_pasta_n_commit(repositorio, pasta_n_commit):
  try:
    os.environ['REPOSITORIO_ORIGINAL'] = repositorio
    os.environ['REPOSITORIO_INICIAL'] = pasta_n_commit
    # Cria uma nova pasta para referenciar apenas os dados do 1o commit
    comando = !cp -R $REPOSITORIO_ORIGINAL $REPOSITORIO_INICIAL
  except Exception as e:
    print(f'Erro: {str(e)}')
  return comando

def checkout_n_commit(pasta_n_commit, n_commit):
  try:
    os.environ['REPOSITORIO_INICIAL'] = pasta_n_commit
    os.environ['PRIMEIRO_COMMIT'] = n_commit
    # git checkout $( git rev-list --all | tail -n 1 )
    comando1 = "cd $REPOSITORIO_INICIAL && git checkout $PRIMEIRO_COMMIT"
    os.system(comando1)
    print(f'Checkout do repositorio no commit {n_commit} realizado com sucesso!')
  except Exception as e:
    print(f'Erro: {str(e)}')

def gera_df_info_repositorio(pasta_n_commit):
  '''
      # Quantidade de arquivos na pasta nome_repositorio
      # Quantidade de commits do repositorio nome_repositorio
      # Tamanho (MB) da pasta nome_repositorio
      # Quantidade de colaboradores (devs que commitaram pelo menos uma vez no projeto nome_repositorio)
      # Data do primeiro commit do projeto nome_repositorio
      # Data do ultimo commit do projeto nome_repositorio
  '''
  try:
    os.environ['REPOSITORIO_INICIAL'] = pasta_n_commit
    qtd_arquivos = !find $REPOSITORIO_INICIAL -type f | wc -l
    qtd_commits = !cd $REPOSITORIO_INICIAL && git log --oneline | grep -c .
    tamanho_pasta_mb = !du -hs --block-size=1M $REPOSITORIO_INICIAL | cut -f1
    qtd_colaboradores_commits = !cd $REPOSITORIO_INICIAL && git log --pretty=format:"%ae" | sort -u | grep -v "noreply.github.com" | wc -l
    data_primeiro_commit =!cd $REPOSITORIO_INICIAL && git log --reverse --pretty=format:%ad --date=short | head -1
    data_ultimo_commit = !cd $REPOSITORIO_INICIAL && git log -1 --pretty=format:%ad --date=short

    dict_info_repo = {
      'nome':pasta_n_commit,
      'qtd_arquivos':qtd_arquivos,
      'qtd_commits':qtd_commits,
      'tamanho_pasta_mb':tamanho_pasta_mb,
      'qtd_colaboradores_commits':qtd_colaboradores_commits,
      'data_primeiro_commit':data_primeiro_commit,
      'data_ultimo_commit':data_ultimo_commit,
    }

    df_info_repo = pd.DataFrame(dict_info_repo)
  except Exception as e:
    print(f'Erro: {str(e)}')
  return df_info_repo

def gera_df_dados(pasta_n_commit):
  lista_linguagens, lista_files, lista_blank, lista_comment, lista_code = list(), list(), list(), list(), list()
  l_temp = []
  vl_temp = []
  os.environ['REPOSITORIO_INICIAL'] = pasta_n_commit
  try:
    # Detalha os tipos de arquivos que aparecem no repositório do nome_repositorio
    relacao_linguagens = !cd $REPOSITORIO_INICIAL && cloc .
    #relacao_linguagens[checa_indice_language(relacao_linguagens):]
    linguagens_encontradas = relacao_linguagens[(checa_indice_language(relacao_linguagens)+2):]
    for linha in linguagens_encontradas:
      elementos = linha.split(' ')
      novos_elementos = []
      linguagem_encontrada = []
      valores_da_linguagem = []
      for item in elementos:
        if item != '':
          novos_elementos.append(item)
      linguagem_encontrada.append(novos_elementos[:-4])
      valores_da_linguagem.append(novos_elementos[-4:])
      l_temp = l_temp + linguagem_encontrada
      vl_temp = vl_temp + valores_da_linguagem

    l_temp = l_temp[:-3]
    vl_temp = vl_temp[:-3]
    for i,each in enumerate(vl_temp):
      lista_files.append(each[0])
      lista_blank.append(each[1])
      lista_comment.append(each[2])
      lista_code.append(each[3])
  except Exception as e:
    print(f'Erro: {str(e)}')
  try:
    dict_dados_linguagens = {
    'linguagens':l_temp,
    'files':lista_files,
    'blank':lista_blank,
    'comment':lista_comment,
    'code':lista_code
    }
    df_dados_linguagens = pd.DataFrame(dict_dados_linguagens)
    df_dados_linguagens['files'] = df_dados_linguagens['files'].apply(lambda x: int(x))
    df_dados_linguagens['blank'] = df_dados_linguagens['blank'].apply(lambda x: int(x))
    df_dados_linguagens['comment'] = df_dados_linguagens['comment'].apply(lambda x: int(x))
    df_dados_linguagens['code'] = df_dados_linguagens['code'].apply(lambda x: int(x))
  except Exception as e2:
    print(f'Erro: {str(e2)}')

  return df_dados_linguagens

def gera_df_dados_repositorios(df_info_repo, df_info_repo_cf):
  df_repo = None
  try:
    l_cf = df_info_repo_cf.columns.tolist()
    c_c1 = df_info_repo.iloc[0].to_list()
    c_cf = df_info_repo_cf.iloc[0].to_list()

    dict_repo= {
        'descricao':l_cf,
        'dados_c1':c_c1,
        'dados_cf':c_cf
    }
    df_repo = pd.DataFrame(dict_repo)
  except Exception as e:
    print(f'Erro: {str(3)}')
  return df_repo

def gera_df_dados_repositorios2(df_info_repo, df_info_repo_cf):
  df_repo = None
  try:
    l_cf = df_info_repo_cf.columns.tolist()
    c_c1 = df_info_repo.iloc[0].to_list()
    c_cf = df_info_repo_cf.iloc[0].to_list()

    dict_repo= {
        'dados_cf':c_cf
    }
    df_repo = pd.DataFrame(dict_repo)
  except Exception as e:
    print(f'Erro: {str(3)}')
  return df_repo

def gera_df_design_smells(pasta_testes, lista):
  path_file = pasta_testes + '/' + 'designCodeSmells.csv'
  df_ds = pd.read_csv(path_file)
  elemento = pasta_testes,df_ds
  lista.append(elemento)
  return df_ds

def is_valid_url(url):
    try:
        response = requests.head(url)
        return response.status_code in range(200, 300)  # Check if status code is in the 2xx range
    except requests.exceptions.RequestException:
        return False  # Any error or exception indicates an invalid or non-existent URL

def get_elements_with_pattern(strings_list, pattern):
    matching_elements = [s for s in strings_list if re.search(pattern, s)]
    return matching_elements

In [ ]:
# Example list of strings
strings_list = ['gson-parent-1.0', 'gson-parent-2.0', 'gson-child-1.0', 'gson-parent-3.0']
pattern_to_match = r'gson-parent'

result = get_elements_with_pattern(strings_list, pattern_to_match)
print(result)

# Prepara DesigniteJava

## Enterprise

In [3]:
url = 'https://www.designite-tools.com/static/download/DJE/DesigniteJava.jar'
download_designite(url)

DesigniteJava.jar downloaded successfully!


In [4]:
!mkdir DesigniteJava
!mv DesigniteJava.jar DesigniteJava

In [6]:
!chmod 775 DesigniteJava

In [5]:
!ls -liath DesigniteJava/DesigniteJava.jar

5636104 -rw-r--r-- 1 root root 24M Jul 19 21:23 DesigniteJava/DesigniteJava.jar


In [7]:
!java -jar DesigniteJava/DesigniteJava.jar -v

DesigniteJava Enterprise. Version 2.4.2
Copyright (C) 2023 Designite. All rights reserved.
Total time (sec): 0


In [8]:
!java -jar DesigniteJava/DesigniteJava.jar -r xxx

DesigniteJava Enterprise. Version 2.4.2
Copyright (C) 2023 Designite. All rights reserved.
Error - License registration unsuccessful. It seems the license key has been already registered.
Total time (sec): 1


## Community

In [9]:
url_community = 'https://www.designite-tools.com/static/download/DJC/DesigniteJava.jar'
download_designite(url_community)

DesigniteJava.jar downloaded successfully!


In [10]:
!mkdir DesigniteJavaCommunity
!mv DesigniteJava.jar DesigniteJavaCommunity

In [11]:
!ls -liath DesigniteJavaCommunity/DesigniteJava.jar

5636113 -rw-r--r-- 1 root root 8.4M Jul 19 21:24 DesigniteJavaCommunity/DesigniteJava.jar


In [12]:
!chmod 775 DesigniteJavaCommunity/DesigniteJava.jar

In [13]:
!ls -liath DesigniteJavaCommunity/DesigniteJava.jar

5636113 -rwxrwxr-x 1 root root 8.4M Jul 19 21:24 DesigniteJavaCommunity/DesigniteJava.jar


# Análise Designite

## Carrega o repositório alvo

In [14]:
!git clone https://github.com/apache/cassandra.git

Cloning into 'cassandra'...
remote: Enumerating objects: 391438, done.
remote: Counting objects: 100% (1944/1944), done.
remote: Compressing objects: 100% (1045/1045), done.
remote: Total 391438 (delta 884), reused 1363 (delta 644), pack-reused 389494
Receiving objects: 100% (391438/391438), 391.12 MiB | 23.24 MiB/s, done.
Resolving deltas: 100% (228080/228080), done.


In [15]:
!cd cassandra && git status

On branch trunk
Your branch is up to date with 'origin/trunk'.

nothing to commit, working tree clean


### Pega a lista de tags

In [16]:
lista_tags = !cd cassandra && git tag

In [17]:
pattern_to_extract = r'cassandra-3.'
get_elements_with_pattern(lista_tags, pattern_to_extract)

['cassandra-3.0.0',
 'cassandra-3.0.0-alpha1',
 'cassandra-3.0.0-beta1',
 'cassandra-3.0.0-beta2',
 'cassandra-3.0.0-rc1',
 'cassandra-3.0.0-rc2',
 'cassandra-3.0.1',
 'cassandra-3.0.10',
 'cassandra-3.0.11',
 'cassandra-3.0.12',
 'cassandra-3.0.13',
 'cassandra-3.0.14',
 'cassandra-3.0.15',
 'cassandra-3.0.16',
 'cassandra-3.0.17',
 'cassandra-3.0.18',
 'cassandra-3.0.19',
 'cassandra-3.0.2',
 'cassandra-3.0.20',
 'cassandra-3.0.21',
 'cassandra-3.0.22',
 'cassandra-3.0.23',
 'cassandra-3.0.24',
 'cassandra-3.0.25',
 'cassandra-3.0.26',
 'cassandra-3.0.27',
 'cassandra-3.0.28',
 'cassandra-3.0.29',
 'cassandra-3.0.3',
 'cassandra-3.0.4',
 'cassandra-3.0.5',
 'cassandra-3.0.6',
 'cassandra-3.0.7',
 'cassandra-3.0.8',
 'cassandra-3.0.9',
 'cassandra-3.1',
 'cassandra-3.1.1',
 'cassandra-3.10',
 'cassandra-3.11.0',
 'cassandra-3.11.1',
 'cassandra-3.11.10',
 'cassandra-3.11.11',
 'cassandra-3.11.12',
 'cassandra-3.11.13',
 'cassandra-3.11.14',
 'cassandra-3.11.15',
 'cassandra-3.11.2',
 

# Análise da tag cassandra-3.0.0

In [18]:
# Cria uma nova pasta para referenciar apenas os dados do 1o commit
commit_tag_300 = get_commit_tag(tag='cassandra-3.0.0', repositorio='cassandra')
cria_pasta_n_commit(repositorio='cassandra', pasta_n_commit='cassandra_3_0_0')
checkout_n_commit('cassandra_3_0_0', commit_tag_300[0])

Checkout do repositorio no commit 96f407bce56b98cd824d18e32ee012dbb99a0286 realizado com sucesso!


In [19]:
!ls -liath cassandra_3_0_0/src/java/

total 12K
5643112 drwxr-xr-x 5 root root 4.0K Jul 19 21:25 ..
5643117 drwxr-xr-x 3 root root 4.0K Jul 19 21:25 .
5643123 drwxr-xr-x 3 root root 4.0K Jul 19 21:25 org


## Análise do DJ Enterprise

In [20]:
!mkdir cassandra_3_0_0/dje

In [21]:
!java -jar DesigniteJava/DesigniteJava.jar -i cassandra_3_0_0/src/java/ -o cassandra_3_0_0/dje

Searching classpath folders ...
Could not find any classpath folder.
DesigniteJava Enterprise. Version 2.4.2
Copyright (C) 2023 Designite. All rights reserved.
Parsing the source code ...
Resolving symbols...
Computing metrics...
Detecting code smells...
The trial version allows you to analyze an application lesser than 50000 lines of code in size. The selected application has crossed the threshold, therefore the detailed results are disabled. 
Please consider buying a Professional edition of the tool for unrestricted access to the functionality.
wrapping up ...
--Analysis summary--
	Total LOC analyzed: 136365	Number of packages: 75
	Number of classes: 2210	Number of methods: 12283
-Total architecture smell instances detected-
	Cyclic dependency: 61	God component: 26
	Ambiguous interface: 0 	Feature concentration: 52
	Unstable dependency: 43	Scattered functionality: 181
	Dense structure: 1
-Total design smell instances detected-
	Imperative abstraction: 5	Multifaceted abstraction: 10
	

In [22]:
!ls -liath cassandra_3_0_0/dje/ArchitectureSmells.csv
!ls -liath cassandra_3_0_0/dje/DesignSmells.csv
!ls -liath cassandra_3_0_0/dje/ImplementationSmells.csv
!ls -liath cassandra_3_0_0/dje/MethodMetrics.csv
!ls -liath cassandra_3_0_0/dje/TestSmells.csv
!ls -liath cassandra_3_0_0/dje/TestabilitySmells.csv
!ls -liath cassandra_3_0_0/dje/TypeMetrics.csv

5644766 -rw-r--r-- 1 root root 64 Jul 19 21:31 cassandra_3_0_0/dje/ArchitectureSmells.csv
5644762 -rw-r--r-- 1 root root 68 Jul 19 21:31 cassandra_3_0_0/dje/DesignSmells.csv
5644765 -rw-r--r-- 1 root root 109 Jul 19 21:31 cassandra_3_0_0/dje/ImplementationSmells.csv
5644761 -rw-r--r-- 1 root root 122 Jul 19 21:31 cassandra_3_0_0/dje/MethodMetrics.csv
5644764 -rw-r--r-- 1 root root 78 Jul 19 21:31 cassandra_3_0_0/dje/TestSmells.csv
5644763 -rw-r--r-- 1 root root 73 Jul 19 21:31 cassandra_3_0_0/dje/TestabilitySmells.csv
5644760 -rw-r--r-- 1 root root 105 Jul 19 21:31 cassandra_3_0_0/dje/TypeMetrics.csv


In [23]:
!zip -r analysis_cassandra.zip cassandra_3_0_0/dje

  adding: cassandra_3_0_0/dje/ (stored 0%)
  adding: cassandra_3_0_0/dje/TestSmells.csv (deflated 22%)
  adding: cassandra_3_0_0/dje/TestabilitySmells.csv (deflated 18%)
  adding: cassandra_3_0_0/dje/ArchitectureSmells.csv (deflated 14%)
  adding: cassandra_3_0_0/dje/ImplementationSmells.csv (deflated 22%)
  adding: cassandra_3_0_0/dje/DesignSmells.csv (deflated 18%)
  adding: cassandra_3_0_0/dje/TypeMetrics.csv (deflated 16%)
  adding: cassandra_3_0_0/dje/DesigniteLog19072023_2125.txt (deflated 35%)
  adding: cassandra_3_0_0/dje/MethodMetrics.csv (deflated 23%)


### Architectural Smells

In [27]:
df_architectural_smells = pd.read_csv('cassandra_3_0_0/dje/ArchitectureSmells.csv')

In [28]:
df_architectural_smells

,Project Name,Package Name,Architecture Smell,Cause of the Smell


## Análise do DJ Community

In [29]:
!mkdir cassandra_3_0_0/djc

In [30]:
!java -jar DesigniteJavaCommunity/DesigniteJava.jar -i cassandra_3_0_0/src/java -o cassandra_3_0_0/djc

Parsing the source code ...
Resolving symbols...
Extracting metrics...
Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.
Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.
Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.
Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due 

In [31]:
!ls -liath cassandra_3_0_0/djc/designCodeSmells.csv

5644772 -rw-r--r-- 1 root root 232K Jul 19 21:40 cassandra_3_0_0/djc/designCodeSmells.csv


In [32]:
df_design_smells = pd.read_csv('cassandra_3_0_0/djc/designCodeSmells.csv')
df_design_smells

,Project Name,Package Name,Type Name,Code Smell
0,java,org.apache.cassandra.repair,SystemDistributedKeyspace,Deficient Encapsulation
1,java,org.apache.cassandra.repair,SystemDistributedKeyspace,Cyclic-Dependent Modularization
2,java,org.apache.cassandra.repair,NodePair,Deficient Encapsulation
3,java,org.apache.cassandra.repair,NodePair,Cyclic-Dependent Modularization
4,java,org.apache.cassandra.repair,NodePairSerializer,Unutilized Abstraction
...,...,...,...,...
3074,java,org.apache.cassandra.thrift,ThriftResultsMerger,Broken Hierarchy
3075,java,org.apache.cassandra.thrift,ThriftResultsMerger,Cyclic-Dependent Modularization
3076,java,org.apache.cassandra.thrift,PartitionMerger,Unutilized Abstraction
3077,java,org.apache.cassandra.thrift,SuperColumnsPartitionMerger,Unutilized Abstraction


In [33]:
df_design_smells['Type Name'].unique()

array(['SystemDistributedKeyspace', 'NodePair', 'NodePairSerializer', ...,
       'ThriftResultsMerger', 'PartitionMerger',
       'SuperColumnsPartitionMerger'], dtype=object)

In [34]:
# Esta lista eh oriunda da analise AMLOC x FOC e das analises dos 3o quartis AMLOC e FOC
#lista_arquivos_criticos = [("StorageService.java", 3537, 144), ("ColumnFamilyStore.java", 1966, 89), ("DatabaseDescriptor.java", 2092, 87), ("CompactionManager.java", 1279, 74), ("StorageProxy.java", 1667, 53), ("SSTableReader.java", 767, 53), ("Config.java", 553, 52), ("CassandraDaemon.java", 1001, 51), ("SelectStatement.java", 1174, 44), ("SinglePartitionReadCommand.java", 994, 38),("NodeProbe.java", 513, 37), ("MessagingService.java", 574, 36)]
lista_classes_criticas = ["StorageService", "ColumnFamilyStore", "DatabaseDescriptor", "CompactionManager", "StorageProxy", "SSTableReader", "Config", "CassandraDaemon", "SelectStatement", "SinglePartitionReadCommand","NodeProbe", "MessagingService"]

In [37]:
pesquisa = df_design_smells['Type Name'].str.contains('|'.join(lista_classes_criticas))

In [38]:
df_critical_classes = df_design_smells[pesquisa]
df_critical_classes

,Project Name,Package Name,Type Name,Code Smell
77,java,org.apache.cassandra.config,DatabaseDescriptor,Unexploited Encapsulation
78,java,org.apache.cassandra.config,DatabaseDescriptor,Missing Hierarchy
79,java,org.apache.cassandra.config,DatabaseDescriptor,Cyclic-Dependent Modularization
80,java,org.apache.cassandra.config,DatabaseDescriptor,Insufficient Modularization
81,java,org.apache.cassandra.config,DatabaseDescriptor,Hub-like Modularization
96,java,org.apache.cassandra.config,Config,Deficient Encapsulation
109,java,org.apache.cassandra.config,YamlConfigurationLoader,Unutilized Abstraction
335,java,org.apache.cassandra.service,StorageService,Deficient Encapsulation
336,java,org.apache.cassandra.service,StorageService,Cyclic-Dependent Modularization
337,java,org.apache.cassandra.service,StorageService,Insufficient Modularization


In [39]:
df_critical_classes.columns = ['ProjectName', 'PackageName', 'TypeName', 'CodeSmell']
df_critical_classes = df_critical_classes.query("CodeSmell=='Cyclic-Dependent Modularization' or CodeSmell=='Hub-like Modularization'")
df_critical_classes

,ProjectName,PackageName,TypeName,CodeSmell
79,java,org.apache.cassandra.config,DatabaseDescriptor,Cyclic-Dependent Modularization
81,java,org.apache.cassandra.config,DatabaseDescriptor,Hub-like Modularization
336,java,org.apache.cassandra.service,StorageService,Cyclic-Dependent Modularization
338,java,org.apache.cassandra.service,StorageService,Hub-like Modularization
366,java,org.apache.cassandra.service,StorageProxy,Cyclic-Dependent Modularization
368,java,org.apache.cassandra.service,StorageProxy,Hub-like Modularization
402,java,org.apache.cassandra.service,CassandraDaemon,Cyclic-Dependent Modularization
706,java,org.apache.cassandra.db,ColumnFamilyStore,Cyclic-Dependent Modularization
708,java,org.apache.cassandra.db,ColumnFamilyStore,Hub-like Modularization
791,java,org.apache.cassandra.db,SinglePartitionReadCommand,Cyclic-Dependent Modularization


In [41]:
df_analise_designite_critical_classes = df_critical_classes.sort_values(by='TypeName')
df_analise_designite_critical_classes.to_csv('cassandra_3_0_0/djc/analise_designite_classes_criticas.csv', index=False)
df_design_smells.to_csv('cassandra_3_0_0/djc/analise_designite.csv', index=False)

# Análise da tag cassandra-3.11.11

In [40]:
# Cria uma nova pasta para referenciar apenas os dados do 1o commit
commit_tag_31111 = get_commit_tag(tag='cassandra-3.11.11', repositorio='cassandra')
cria_pasta_n_commit(repositorio='cassandra', pasta_n_commit='cassandra_3_11_11')
checkout_n_commit('cassandra_3_11_11', commit_tag_31111[0])

Checkout do repositorio no commit 4cafe2288e56e1135d65e76adbcd6c2de9306d6b realizado com sucesso!


In [41]:
!mkdir cassandra_3_11_11/djc

In [42]:
!java -jar DesigniteJavaCommunity/DesigniteJava.jar -i cassandra_3_11_11/src/java -o cassandra_3_11_11/djc

Parsing the source code ...
Resolving symbols...
Extracting metrics...
Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.Inheritance depth analysis step skipped due to memory overflow.
Inheritance depth analysis step skipped due to memory overflow.
Inheritance depth analysis step skipped due to memory overflow.
Inheritance depth analysis step skipped due to memory overflow.
Inheritance depth analysis step skipped due to memory overflow.
Inheritance depth analysis step skipped due to memory overflow.
Inheritance depth analysis step skipped due to memory overflow.
Inheritance depth analysis step skipped due to memory overflow.
Inheritance depth analysis step skipped due to memory overflow.
Inheritance depth analysis step skipped due to memory overflow.
Inheritance depth analysis step skipped due to memory overflow.
Inheritance depth analysis step skipped due to memory overflow.
Inheritance depth analysis step ski

In [43]:
df_design_smells_final = pd.read_csv('cassandra_3_11_11/djc/designCodeSmells.csv')
df_design_smells_final

,Project Name,Package Name,Type Name,Code Smell
0,java,org.apache.cassandra.repair,SystemDistributedKeyspace,Deficient Encapsulation
1,java,org.apache.cassandra.repair,SystemDistributedKeyspace,Cyclic-Dependent Modularization
2,java,org.apache.cassandra.repair,NodePair,Deficient Encapsulation
3,java,org.apache.cassandra.repair,NodePair,Cyclic-Dependent Modularization
4,java,org.apache.cassandra.repair,NodePairSerializer,Unutilized Abstraction
...,...,...,...,...
3684,java,org.apache.cassandra.thrift,ThriftResultsMerger,Broken Hierarchy
3685,java,org.apache.cassandra.thrift,ThriftResultsMerger,Cyclic-Dependent Modularization
3686,java,org.apache.cassandra.thrift,PartitionMerger,Unutilized Abstraction
3687,java,org.apache.cassandra.thrift,SuperColumnsPartitionMerger,Unutilized Abstraction


In [44]:
pesquisa_final = df_design_smells_final['Type Name'].str.contains('|'.join(lista_classes_criticas))

In [45]:
df_critical_classes_final = df_design_smells_final[pesquisa_final]
df_critical_classes_final.columns = ['ProjectName', 'PackageName', 'TypeName', 'CodeSmell']
df_critical_classes_final = df_critical_classes_final.query("CodeSmell=='Cyclic-Dependent Modularization' or CodeSmell=='Hub-like Modularization'")

In [46]:
df_critical_classes_final

,ProjectName,PackageName,TypeName,CodeSmell
77,java,org.apache.cassandra.config,DatabaseDescriptor,Cyclic-Dependent Modularization
79,java,org.apache.cassandra.config,DatabaseDescriptor,Hub-like Modularization
96,java,org.apache.cassandra.config,Config,Cyclic-Dependent Modularization
384,java,org.apache.cassandra.service,StorageService,Cyclic-Dependent Modularization
386,java,org.apache.cassandra.service,StorageService,Hub-like Modularization
418,java,org.apache.cassandra.service,StorageProxy,Cyclic-Dependent Modularization
420,java,org.apache.cassandra.service,StorageProxy,Hub-like Modularization
457,java,org.apache.cassandra.service,CassandraDaemon,Cyclic-Dependent Modularization
778,java,org.apache.cassandra.db,ColumnFamilyStore,Cyclic-Dependent Modularization
780,java,org.apache.cassandra.db,ColumnFamilyStore,Hub-like Modularization


In [47]:
df_analise_designite_critical_classes_final = df_critical_classes_final.sort_values(by='TypeName')
df_analise_designite_critical_classes_final.to_csv('cassandra_3_11_11/djc/analise_designite_classes_criticas.csv', index=False)
df_design_smells_final.to_csv('cassandra_3_11_11/djc/analise_designite.csv', index=False)

In [48]:
df_critical_classes_final['TypeName'].unique()

array(['DatabaseDescriptor', 'Config', 'StorageService', 'StorageProxy',
       'CassandraDaemon', 'ColumnFamilyStore',
       'SinglePartitionReadCommand', 'ColumnFamilyStoreCQLHelper',
       'Configuration', 'CompactionManager', 'AuthConfig',
       'MessagingService', 'SSTableReader', 'ConfiguredLimit',
       'NodeProbe', 'SelectStatement'], dtype=object)